In [1]:
from pyyoutube import Api



ModuleNotFoundError: No module named 'pyyoutube'

In [ ]:
!2+2

In [ ]:
import os
import logging
import yaml
from pyyoutube import Api
import json
import requests
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
import re
import numpy as np
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import SpectralClustering
from sklearn.metrics import f1_score, silhouette_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.tracking import MlflowClient

In [ ]:
def get_data(YOUTUBE_API_KEY, videoId, maxResults, nextPageToken):
    """
    Получение информации со страницы с видео
    """
    YOUTUBE_URI = 'https://www.googleapis.com/youtube/v3/commentThreads?key={KEY}&textFormat=plainText&' + \
        'part=snippet&videoId={videoId}&maxResults={maxResults}&pageToken={nextPageToken}'
    format_youtube_uri = YOUTUBE_URI.format(KEY=YOUTUBE_API_KEY,
                                            videoId=videoId,
                                            maxResults=maxResults,
                                            nextPageToken=nextPageToken)
    content = requests.get(format_youtube_uri).text
    data = json.loads(content)
    return data


def get_text_of_comment(data):
    """
    Получение комментариев из полученных данных под одним видео
    """
    comms = set()
    for item in data['items']:
        comm = item['snippet']['topLevelComment']['snippet']['textDisplay']
        comms.add(comm)
    return comms


def get_all_comments(YOUTUBE_API_KEY, query, count_video=10, limit=30, maxResults=10, nextPageToken=''):
    """
    Выгрузка maxResults комментариев
    """
    api = Api(api_key=YOUTUBE_API_KEY)
    video_by_keywords = api.search_by_keywords(q=query,
                                               search_type=["video"],
                                               count=count_video,
                                               limit=limit)
    videoId = [x.id.videoId for x in video_by_keywords.items]

    comments_all = []
    for id_video in videoId:
        try:
            data = get_data(YOUTUBE_API_KEY,
                            id_video,
                            maxResults=maxResults,
                            nextPageToken=nextPageToken)
            comment = list(get_text_of_comment(data))
            comments_all.append(comment)
        except:
            continue
    comments = sum(comments_all, [])
    return comments

In [ ]:
!ls ../config

In [ ]:
config_path = os.path.join('../config/params_all.yaml')
config = yaml.safe_load(open(config_path))['train']

In [ ]:
SEED = config['SEED']

In [ ]:
config

In [ ]:
comments = get_all_comments(**config['comments'])

In [ ]:
comments[:10]

In [ ]:
def remove_emoji(string):
    """
    Удаление эмоджи из текста
    """
    emoji_pattern = re.compile("["u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def remove_links(string):
    """
    Удаление ссылок
    """
    string = re.sub(r'http\S+', '', string)  # remove http links
    string = re.sub(r'bit.ly/\S+', '', string)  # rempve bitly links
    string = re.sub(r'www\S+', '', string)  # rempve bitly links
    string = string.strip('[link]')  # remove [links]
    return string


def preprocessing(string, stopwords, stem):
    """
    Простой препроцессинг текста, очистка, лематизация, удаление коротких слов
    """
    string = remove_emoji(string)
    string = remove_links(string)

    # удаление символов "\r\n"
    str_pattern = re.compile("\r\n")
    string = str_pattern.sub(r'', string)

    # очистка текста от символов
    string = re.sub('(((?![а-яА-Я ]).)+)', ' ', string)
    # лематизация
    string = ' '.join([
        re.sub('\\n', '', ' '.join(stem.lemmatize(s))).strip()
        for s in string.split()
    ])
    # удаляем слова короче 3 символов
    string = ' '.join([s for s in string.split() if len(s) > 3])
    # удаляем стоп-слова
    string = ' '.join([s for s in string.split() if s not in stopwords])
    return string


def get_clean_text(data, stopwords):
    """
    Получение текста в преобразованной после очистки
    матричном виде, а также модель векторизации
    """
    # Простой препроцессинг текста
    stem = Mystem()
    comments = [preprocessing(x, stopwords, stem) for x in data]
    # Удаление комментов, которые имеют меньше, чем 5 слов
    comments = [y for y in comments if len(y.split()) > 5]
    #common_texts = [i.split(' ') for i in comments]
    return comments


def vectorize_text(data, tfidf):
    """
    Получение матрицы кол-ва слов в комменариях
    Очистка от пустых строк
    """
    # Векторизация
    X_matrix = tfidf.transform(data).toarray()
    # Удаляем строки в матрице с пустыми значениями
    mask = (np.nan_to_num(X_matrix) != 0).any(axis=1)
    return X_matrix[mask]

In [ ]:
comments_clean = get_clean_text(comments, stopwords.words(config['stopwords']))
tfidf = TfidfVectorizer(**config['tf_model']).fit(comments_clean)

In [ ]:
config

In [ ]:
comments_clean[:10]

In [ ]:
X_matrix = vectorize_text(comments_clean, tfidf)

In [ ]:
X_matrix.shape

In [ ]:
tfidf.get_feature_names()[:10]

In [ ]:
def get_clusters(data, count_max_clusters, random_state, affinity,
                 silhouette_metric):
    """
    Подбор наилучшего числа кластеров, возвращает полученные кластера тематик
    """
    cluster_labels = {}
    silhouette_mean = []

    for i in range(2, count_max_clusters, 1):
        clf = SpectralClustering(n_clusters=i,
                                 affinity=affinity,
                                 random_state=random_state)
        #clf = KMeans(n_clusters=n, max_iter=1000, n_init=1)
        clf.fit(data)
        labels = clf.labels_
        cluster_labels[i] = labels
        silhouette_mean.append(
            silhouette_score(data, labels, metric=silhouette_metric))
    n_clusters = silhouette_mean.index(max(silhouette_mean)) + 2
    return cluster_labels[n_clusters]


def get_f1_score(y_test, y_pred, unique_cluster_labels):
    """
    Возращает результат обучения классификатора по тематикам
    """
    return f1_score(
        y_test, y_pred,
        average='macro') \
        if len(unique_cluster_labels) > 2 \
        else f1_score(y_test, y_pred)

In [ ]:
cluster_labels = get_clusters(X_matrix,
                                 random_state=SEED,
                                 **config['clustering'])

In [ ]:
config

In [ ]:
cluster_labels[:10]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_matrix,
                                                    cluster_labels,
                                                    **config['cross_val'],
                                                    random_state=SEED)

In [ ]:
clf_lr = LogisticRegression(**config['model'])

In [ ]:
%%bash
export MLFLOW_REGISTRY_URI=../mlflow

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(config['name_experiment'])
with mlflow.start_run():
    clf_lr.fit(X_train, y_train)
    print(clf_lr.predict_proba(X_test))

    # Логирование модели и параметров
    mlflow.log_param(
        'f1', get_f1_score(y_test, clf_lr.predict(X_test),
                           set(cluster_labels)))
    mlflow.sklearn.log_model(
        tfidf,
        artifact_path="vector",
        registered_model_name=f"{config['model_vec']}")
    mlflow.sklearn.log_model(
        clf_lr,
        artifact_path='model_lr',
        registered_model_name=f"{config['model_lr']}")
    mlflow.end_run()

In [ ]:
mlflow.get_artifact_uri()

In [ ]:
def get_version_model(config_name, client):
    """
    Получение последней версии модели из MLFlow
    """
    dict_push = {}
    for count, value in enumerate(
        client.search_model_versions(f"name='{config_name}'")):
        # client.list_registered_models()):
        # Все версии модели
        dict_push[count] = value
    return dict(list(dict_push.items())[-1][1])['version']

In [ ]:
client = MlflowClient()
last_version_lr = get_version_model(config['model_lr'], client)
last_version_vec = get_version_model(config['model_vec'], client)

In [ ]:
last_version_lr

In [ ]:
last_version_vec